In [1]:
pip install python-pptx


   ---------------------------------------- 0.0/471.6 kB ? eta -:--:--
   -- ------------------------------------- 30.7/471.6 kB 1.4 MB/s eta 0:00:01
   --------- ------------------------------ 112.6/471.6 kB 1.3 MB/s eta 0:00:01
   ----------------- ---------------------- 204.8/471.6 kB 1.6 MB/s eta 0:00:01
   ----------------------- ---------------- 276.5/471.6 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 471.6/471.6 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.8 MB 9.9 MB/s eta 0:00:01
   ------- -------------------------------- 0.7/3.8 MB 8.8 MB/s eta 0:00:01
   --------- ------------------------------ 0.9/3.8 MB 7.5 MB/s eta 0:00:01
   ------------- -------------------------- 1.3/3.8 MB 7.4 MB/s eta 0:00:01
   ------------------ --------------------- 1.8/3.8 MB 8.0 MB/s eta 0:00:01
   --------------------- ------------------ 2.0/3.8 MB 7.5 MB/s eta 0:00:01
   --


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
from pptx import Presentation
from pptx.util import Inches, Pt
from io import BytesIO

def copy_slide(source_slide, dest_presentation):
    slide_layout = dest_presentation.slide_layouts[5]  # Use blank layout
    new_slide = dest_presentation.slides.add_slide(slide_layout)

    for shape in source_slide.shapes:
        if shape.shape_type == 13:  # Picture
            image_stream = BytesIO(shape.image.blob)
            left = shape.left
            top = shape.top
            width = shape.width
            height = shape.height
            new_slide.shapes.add_picture(image_stream, left, top, width, height)
        else:
            if shape.shape_type == 1:  # AutoShape
                new_shape = new_slide.shapes.add_shape(
                    shape.auto_shape_type,
                    shape.left, shape.top,
                    shape.width, shape.height
                )
                if hasattr(shape, "text"):
                    new_shape.text = shape.text
                if shape.fill:
                    new_shape.fill.solid()
                    new_shape.fill.fore_color.rgb = shape.fill.fore_color.rgb
                if shape.line:
                    new_shape.line.color.rgb = shape.line.color.rgb
            elif shape.shape_type == 14:  # TextBox
                new_shape = new_slide.shapes.add_textbox(
                    shape.left, shape.top, shape.width, shape.height
                )
                if hasattr(shape, "text"):
                    new_shape.text = shape.text
                if shape.text_frame:
                    for i, p in enumerate(shape.text_frame.paragraphs):
                        new_p = new_shape.text_frame.add_paragraph()
                        new_p.text = p.text
                        new_p.font.size = p.font.size
                        new_p.font.bold = p.font.bold
                        new_p.font.italic = p.font.italic
                        new_p.font.color.rgb = p.font.color.rgb

def merge_ppt(ppt1_path, ppt2_path, output_path):
    # Load the presentations
    ppt1 = Presentation(ppt1_path)
    ppt2 = Presentation(ppt2_path)

    # Create a new presentation for the output
    merged_ppt = Presentation()

    # Copy slides from the first presentation
    for slide in ppt1.slides:
        copy_slide(slide, merged_ppt)

    # Copy slides from the second presentation
    for slide in ppt2.slides:
        copy_slide(slide, merged_ppt)

    # Save the merged presentation
    merged_ppt.save(output_path)


# Usage example
merge_ppt('C:/Users/user/Desktop/PPT/P1.pptx', 'C:/Users/user/Desktop/PPT/P2.pptx', 'C:/Users/user/Desktop/PPT/merged_ppt.pptx')


In [26]:
from pptx import Presentation
from pptx.util import Inches
from io import BytesIO

def copy_slide(source_presentation, source_slide, dest_presentation):
    slide_layout = dest_presentation.slide_layouts[5]  # Use blank layout
    new_slide = dest_presentation.slides.add_slide(slide_layout)
    
    for shape in source_slide.shapes:
        if shape.shape_type == 13:  # Picture
            image_stream = BytesIO(shape.image.blob)
            new_slide.shapes.add_picture(image_stream, shape.left, shape.top, shape.width, shape.height)
        elif shape.has_text_frame:
            new_shape = new_slide.shapes.add_shape(
                shape.auto_shape_type, shape.left, shape.top, shape.width, shape.height
            )
            new_shape.text = shape.text_frame.text
            for i, p in enumerate(shape.text_frame.paragraphs):
                new_shape.text_frame.paragraphs[i].font.size = p.font.size
                new_shape.text_frame.paragraphs[i].font.bold = p.font.bold
                new_shape.text_frame.paragraphs[i].font.italic = p.font.italic
                new_shape.text_frame.paragraphs[i].font.underline = p.font.underline
                new_shape.text_frame.paragraphs[i].font.color.rgb = p.font.color.rgb
        else:
            new_element = shape.element
            new_slide.shapes._spTree.insert_element_before(new_element, 'p:extLst')

def merge_ppt(ppt1_path, ppt2_path, output_path):
    ppt1 = Presentation(ppt1_path)
    ppt2 = Presentation(ppt2_path)

    merged_ppt = Presentation()

    for slide in ppt1.slides:
        copy_slide(ppt1, slide, merged_ppt)
    for slide in ppt2.slides:
        copy_slide(ppt2, slide, merged_ppt)

    merged_ppt.save(output_path)

# Usage example
merge_ppt('C:/Users/user/Desktop/PPT/P1.pptx', 'C:/Users/user/Desktop/PPT/P2.pptx', 'C:/Users/user/Desktop/PPT/merged_ppt.pptx')
